HW 1 : IMDB Top Animation

In [ ]:
library(rvest)
library(tidyr)
library(stringr)
library(dplyr)

In [ ]:
url <- "https://www.imdb.com/search/title/?genres=animation&groups=top_100"

movie_name <- url %>%
  read_html() %>%
  html_elements("h3.lister-item-header") %>%
  html_text2() %>%
  str_replace("^\\d+\\.\\s+", "")

movie_time <- url %>%
  read_html() %>%
  html_elements("span.runtime") %>%
  html_text2()

movie_genre <- url %>%
  read_html() %>%
  html_elements("span.genre") %>%
  html_text2()

movie_rating <- url %>%
  read_html() %>%
  html_elements("div.ratings-imdb-rating") %>%
  html_text2() %>%
  as.numeric()

movie_votes <- url %>%
  read_html() %>%
  html_elements("p.sort-num_votes-visible") %>%
  html_text2()

imdb_df <- data.frame(movie_name,
                      movie_time,
                      movie_genre,
                      movie_rating,
                      movie_votes)

#manipulate data form column votes
imdb_df <- imdb_df %>%
  mutate(
    movie_name = movie_name,
    movie_time = movie_time,
    movie_genre = movie_genre,
    movie_rating = movie_rating,
    votes = str_match(movie_votes,"Votes:\\s*(\\d+,?\\d+,?\\d+)")[, 2],
    gross = str_match(movie_votes,"Gross:\\s*\\$(\\d+\\.\\d+)")[, 2],
    tops = str_match(movie_votes,"Top\\s*250:\\s*\\#(\\d+)")[, 2]
  ) %>%
  select(movie_name, movie_time, movie_genre, movie_rating, votes, gross, tops)
# found missing value in row 9

r <- regexpr("\\d{4}", movie_name)
imdb_df <- imdb_df %>%
  #separate(movie_votes, sep="\\|", into=c("votes", "gross", "tops")) %>%
  #separate(tops, sep="#", into=c(NA, "tops"), convert = T) %>%
  mutate(year = regmatches(movie_name, r))

#movie_name %>%
  #str_remove("%([0-9]{4})%")
#imdb_df$movie_name <- gsub("%([0-9]{4})%", "", imdb_df$movie_name)

In [ ]:
imdb_df

movie_name,movie_time,movie_genre,movie_rating,votes,gross,tops,year
<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
Spirited Away: De reis van Chihiro (2001),125 min,"Animation, Adventure, Family",8.6,"774,151",10.06,31,2001
The Lion King (1994),88 min,"Animation, Adventure, Drama",8.5,"1,069,389",422.78,36,1994
Spider-Man: Een nieuw universum (2018),117 min,"Animation, Action, Adventure",8.4,"543,123",190.24,66,2018
Toy Story (1995),81 min,"Animation, Adventure, Comedy",8.3,"1,003,871",191.80,74,1995
Coco (I) (2017),105 min,"Animation, Adventure, Comedy",8.4,"519,612",209.73,73,2017
WALL-E (2008),98 min,"Animation, Adventure, Family",8.4,"1,127,153",223.81,57,2008
Your Name (2016),106 min,"Animation, Drama, Fantasy",8.4,"275,602",5.02,82,2016
Toy Story 3 (2010),103 min,"Animation, Adventure, Comedy",8.3,"844,312",415.00,87,2010
Grave of the Fireflies (1988),89 min,"Animation, Drama, War",8.5,"281,765",NA,45,1988


HW 03 : Goodreads Top 100 Literary Novels of All Time

In [ ]:
library(rvest)
library(tidyr)
library(stringr)

In [ ]:
## import data
url_literary <- "https://www.goodreads.com/list/show/13086.Goodreads_Top_100_Literary_Novels_of_All_Time"

In [ ]:
## scraping data
book_top <- url_literary %>%
  read_html() %>%
  html_elements("td.number") %>%
  html_text2()

book_titles <- url_literary %>%
  read_html() %>%
  html_elements("a.bookTitle") %>%
  html_text2()

book_authors <- url_literary %>%
  read_html() %>%
  html_elements("div.authorName__container") %>%
  html_text2()

book_rating <- url_literary %>%
  read_html() %>%
  html_elements("span.minirating") %>%
  html_text2()



In [ ]:
## build data frame
df_literary <- data.frame(
                book_top,
                book_titles,
                book_authors,
                book_rating
)

In [ ]:
head(df_literary)

,book_top,book_titles,book_authors,book_rating
,<chr>,<chr>,<chr>,<chr>
1,1,Pride and Prejudice,Jane Austen,"4.28 avg rating — 3,917,621 ratings"
2,2,1984,George Orwell,"4.19 avg rating — 4,170,217 ratings"
3,3,The Great Gatsby,F. Scott Fitzgerald,"3.93 avg rating — 4,809,543 ratings"
4,4,Jane Eyre,Charlotte Brontë,"4.14 avg rating — 1,965,552 ratings"
5,5,Crime and Punishment,Fyodor Dostoevsky,"4.25 avg rating — 813,773 ratings"
6,6,Lolita,Vladimir Nabokov,"3.88 avg rating — 790,103 ratings"


In [ ]:
## remove ","
df_literary$book_rating <- df_literary$book_rating %>%
  str_remove(",") %>%
  str_remove(",")


In [ ]:
head(df_literary)

,book_top,book_titles,book_authors,book_rating,avg_rating
,<chr>,<chr>,<chr>,<chr>,<dbl>
1,1,Pride and Prejudice,Jane Austen,4.28 avg rating — 3917621 ratings,4.28
2,2,1984,George Orwell,4.19 avg rating — 4170217 ratings,4.19
3,3,The Great Gatsby,F. Scott Fitzgerald,3.93 avg rating — 4809543 ratings,3.93
4,4,Jane Eyre,Charlotte Brontë,4.14 avg rating — 1965552 ratings,4.14
5,5,Crime and Punishment,Fyodor Dostoevsky,4.25 avg rating — 813773 ratings,4.25
6,6,Lolita,Vladimir Nabokov,3.88 avg rating — 790103 ratings,3.88


In [ ]:
# clean data
# extract average rating
df_literary$avg_rating <- df_literary$book_rating %>%
  str_extract("([0-9].[0-9]{2})") %>%
  as.numeric

# extract total rating
df_literary$total_rating <- df_literary$book_rating %>%
  str_extract("[— ]([0-9]+)") %>%
  str_trim() %>%
  as.numeric

## remove rating column
df_literary$book_rating <- NULL

In [ ]:
head(df_literary)

,book_top,book_titles,book_authors,avg_rating,total_rating
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,1,Pride and Prejudice,Jane Austen,4.28,3917621
2,2,1984,George Orwell,4.19,4170217
3,3,The Great Gatsby,F. Scott Fitzgerald,3.93,4809543
4,4,Jane Eyre,Charlotte Brontë,4.14,1965552
5,5,Crime and Punishment,Fyodor Dostoevsky,4.25,813773
6,6,Lolita,Vladimir Nabokov,3.88,790103
